In [24]:
from databases import BDS, PosicaoDm1, Bawm,Crm,BaseExtrato,off_Reliance
from funcoes_datas import FuncoesDatas
from dateutil.relativedelta import relativedelta
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import datetime
from datetime import date
import numpy as np
import math
from emailer import Email, EmailLer
from pretty_html_table import build_table
from funcoes import CalculosRiscoPort

In [25]:
#Chama as classes do databases e as classes do funções data
rel_off_p = off_Reliance('Primeny')
rel_off_c = off_Reliance('Cubo')
crm = Crm()
bawm = Bawm()
datas = FuncoesDatas()
extrato = BaseExtrato()
bds = BDS()
fdt = FuncoesDatas()
calc = CalculosRiscoPort(offshore = True)

In [26]:
cadastro = bawm.po_cadastro_all()
cadastro = cadastro[['NomeContaCRM','Officer','OfficerUsuarioU','Controller','ControllerUsuarioU','ControllerBKP','ControllerBKPUsuarioU','DPM','DPMUsuarioU', 'DeputyDPM', 'DeputyDPMUsuarioU']]
emails_usuarios = crm.lista_usuarios_u()
cadastro['OfficerUsuarioU'] =cadastro['OfficerUsuarioU'].str.lower()
cadastro['ControllerUsuarioU'] = cadastro['ControllerUsuarioU'].str.lower()
cadastro['ControllerBKPUsuarioU'] = cadastro['ControllerBKPUsuarioU'].str.lower()
cadastro['DPMUsuarioU'] = cadastro['DPMUsuarioU'].str.lower()
cadastro['DeputyDPMUsuarioU'] = cadastro['DeputyDPMUsuarioU'].str.lower()

In [27]:
#Procura os parametros para rodar as funções

#Puxando a ultima data do mes anterior
data = bds.banco.hoje()
data = datetime.datetime.date(data)
hoje = fdt.workday(data, n_dias=-4)

#procura a alocação ideal de cada cliente (qual alocação do portfolio modelo?)
po_cadastro = bawm.po_cadastro(apenas_fundos = False)
po_cadastro = po_cadastro.drop(columns=['CodigoProduto'])
alocacao_por_sc = crm.alocacao_ideial_off()

#Traz os fundos da JBFO (prioridade na aplicação das movimentações)
fundos_jbfo = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Fundos_Casa')
fundos_jbfo = fundos_jbfo[['NomeDoProduto','CodigoProduto','Classificação']]
fundos_jbfo['Fundo']=fundos_jbfo['NomeDoProduto'].str.lower()

#Traz os de_paras de mandatos (mandato anterior para o atual)
mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_mandato')
mandatos = mandatos[['Anterior','Novo']]
#mandatos = dict(mandatos.values)
    
# #Traz os de_paras de classificação gps para asset Allocation
dicio_classificacoes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classificacoes=dicio_classificacoes[['Classificação GPS','De Para - Asset Allocation']]
dicio_classificacoes = dict(dicio_classificacoes.values)

#Novos mandatos
novos_mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Novos_mandatos')
novos_mandatos= novos_mandatos[['Portfolio','Classe','Tático']]

#Traz a PO_Cadastro para identificar os responsaveis por cada carteira
cadastro = bawm.po_cadastro_all()[['NomeContaCRM','NomeSuperCarteiraCRM','Officer','OfficerUsuarioU','ControllerUsuarioU','ControllerBKPUsuarioU','DPMUsuarioU','DeputyDPMUsuarioU','TipoCliente']]

#Informa os clientes que estão em distrato
distrato = crm.clientes_em_distrato_off()

#Traz todos os emails pelo usuario U
emails_usuarios = crm.lista_usuarios_u()

#Trazer os produtos GPS e segregar o que é imobiliario (caso precise segregar o que é Reits / ILiquidos)
produtos_gps = crm.produtos_off()
produtos_gps =produtos_gps[['name','new_idmysql','new_tickerid','new_isin','new_idbds','new_idsistemaorigem','new_indgroupaxysidname']].rename(columns={'name':'Ativo','new_idmysql':'IdMysqlProduto','new_tickerid':'Ticker','new_isin':'ISIN','new_idsistemaorigem':'CodigoProduto'}).reset_index()
produtos_gps['ISIN'] = produtos_gps['ISIN'].str.lower()

#Trazendo o De Para de ativos e separando os dataframes para unir com a carteira
de_para_ativos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Tb De Para Subclass')

# #Trazendo todos os produtos da base Reliance
# produtos_rel= rel_off_p.produtos_off().drop(columns=['subclasse'])
# produtos_rel['ISIN'] = produtos_rel['ISIN'].str.lower()

#Trazendo o dicionario da classificação dos ativos dado pela area de investimentos (produtos reliance)
dicio_classificacao_investimentos_rel = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classificacao_investimentos_rel=dicio_classificacao_investimentos_rel[['Classificação Reliance-Investimentos','Classificação GPS - v1']]
dicio_classificacao_investimentos_rel = dict(dicio_classificacao_investimentos_rel.values)

#Dicionario Reliance Classe + Subclasse + Categoria
dicio_classe_sub_categoria = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classe_sub_categoria=dicio_classe_sub_categoria[['Classe_Subclasse_Categoria-Sistema','Classificação GPS - v2']]
dicio_classe_sub_categoria = dict(dicio_classe_sub_categoria.values)

#Saber a qual classe pertence cada subclasse
dicio_classe_subclasse_gps = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classe_subclasse_gps=dicio_classe_subclasse_gps[['Subclasse GPS','Classe']]
dicio_classe_subclasse_gps = dict(dicio_classe_subclasse_gps.values)

# #Classificação de ativos novos ou que não possuem um depara claro
# ativos_novos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Ativos Não Classificados')
# ativos_novos=ativos_novos[['Nome_produto','Classificação GPS']]
# ativos_novos['Classificação GPS']=ativos_novos['Classificação GPS'].str.lower()

#Benchmarks
benchs = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
benchs=benchs[['De Para - Asset Allocation','Index_Bawm']]

In [15]:
def arruma_espaco(x):
    while x[-1]==' ':
        x=x[:-1]
    return x    
    

In [16]:
def formatar(valor):
    int_part, dec_part = format(valor, ',.2f').split('.')
    return f"{int_part.replace(',', '.')},{dec_part}"

In [17]:
def trazer_pu_atual(id_serie):
    try:
        df = bds.serie_historico(idserie=id_serie, intervalo=1)
        if df.empty:
            df = bds.serie_historico(idserie=id_serie, intervalo=4)
            cota = df.iloc[-1,5]
            ultima_data = df.iloc[-1,3]
            ultima_data =datetime.datetime.date(ultima_data)          
        else:
            df = bds.serie_historico(idserie=id_serie, intervalo=1)
            cota = df.iloc[-1,5]
            ultima_data = df.iloc[-1,3]
            ultima_data =datetime.datetime.date(ultima_data)
    except:
        cota=np.nan
        ultima_data = np.nan
    return cota, ultima_data    
        

In [18]:
#Função que identifica clientes inativos
def inativo(x):
    x = x.replace('(','').replace(')','')
    x = x.replace(' ','')
    x = x.strip()
    if 'inativ' in x.lower():
        valor='inativo'
    else:
        valor='ok'
    return valor    

In [28]:
#Consolida a posição atual versus a ideial
alocacao_por_sc = pd.pivot_table(data=alocacao_por_sc, values = 'new_percentual',columns=['new_classedeinvestimentoidname'],index=['new_name','new_conta_supercarteiraidname','new_politicadeinvestimentoidname']).reset_index()
alocacao_por_sc['new_name']=alocacao_por_sc['new_name'].str.lower()
alocacao_por_sc = pd.merge(left = alocacao_por_sc, right = po_cadastro,left_on ='new_conta_supercarteiraidname',right_on = 'NomeSuperCarteiraCRM',how='left')
alocacao_por_sc['inativo']=alocacao_por_sc['new_name'].apply(lambda x: inativo(x))
alocacao_por_sc=alocacao_por_sc[alocacao_por_sc['inativo']!='inativo']
alocacao_por_sc = alocacao_por_sc[['new_name','new_conta_supercarteiraidname','Alt Crédito', 'Alt Multistr','Alt RV L/S', 'Alt Trad/Macro', 'Alt Valor Rel', 'Commodities','Imobiliário', 'Outros', 'P. Equity', 'RF CP', 'RF Glob', 'RF Infl','RF Livre -', 'RF Outros -', 'RV', 'RV Outros -','new_politicadeinvestimentoidname','Titularidade','VetoPowerGestaoTatica','DPM','OfficerUsuarioU','ControllerUsuarioU']].reset_index(drop=True)


In [29]:
alocacao_por_sc

,new_name,new_conta_supercarteiraidname,Alt Crédito,Alt Multistr,Alt RV L/S,Alt Trad/Macro,Alt Valor Rel,Commodities,Imobiliário,Outros,...,RF Livre -,RF Outros -,RV,RV Outros -,new_politicadeinvestimentoidname,Titularidade,VetoPowerGestaoTatica,DPM,OfficerUsuarioU,ControllerUsuarioU
0,@5abe2,AbercrombieAPJ,5.0,NaN,10.0,10.0,NaN,0.0,0.0,NaN,...,10.0,0.0,0.0,0.0,PI AbercrombieA - Offshore,NaN,NaN,NaN,NaN,NaN
1,@5abe3,AbercrombieW,NaN,29.75,NaN,NaN,NaN,0.0,1.5,NaN,...,18.0,0.0,12.5,0.0,Vol 4%S - Offshore,NaN,NaN,NaN,NaN,NaN
2,@5abm1,ArgentinoA,NaN,29.75,NaN,NaN,NaN,0.0,1.5,NaN,...,18.0,0.0,12.5,0.0,Vol 4%S - Offshore,None,Indefinido,NA,u44820,u50836
3,@5aca1,Acacia,5.0,NaN,7.5,10.0,NaN,0.0,1.5,NaN,...,15.0,0.0,12.5,0.0,Vol 4% - Offshore,NaN,NaN,NaN,NaN,NaN
4,@5ada1,Camapua,2.5,NaN,3.5,5.0,NaN,0.0,0.0,NaN,...,32.5,0.0,6.0,0.0,Vol 3% - Offshore,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,@7seqfat,Sequoia,2.0,NaN,3.5,8.0,NaN,NaN,3.5,0.0,...,13.0,NaN,20.0,NaN,PI - Sequoia Offshore,NaN,NaN,NaN,NaN,NaN
1228,@7seqmap,Sequoia,2.0,NaN,3.5,8.0,NaN,NaN,3.5,0.0,...,13.0,NaN,20.0,NaN,PI - Sequoia Offshore,NaN,NaN,NaN,NaN,NaN
1229,@7ski2,Skipper,5.0,NaN,7.5,10.0,NaN,0.0,1.5,NaN,...,15.0,0.0,12.5,0.0,Vol 4% - Offshore,NaN,NaN,NaN,NaN,NaN
1230,@7sup2,Super,5.0,NaN,7.5,10.0,NaN,0.0,1.5,NaN,...,15.0,0.0,12.5,0.0,Vol 4% - Offshore,NaN,NaN,NaN,NaN,NaN


In [22]:
# #Mapeando as informações da base Reliance e linkando com os produtos da base (classificação, isin e ticker do CRM).

#Traz a ultima data do mes anterior
ultimo_extrato = data
ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
ultimo_extrato = fdt.workday(data=ultimo_extrato, n_dias=-1)

carteira_rel = rel_off_p.carteira_reliance(ultimo_extrato)

if carteira_rel.empty:
    ultimo_extrato_rel = ultimo_extrato
    ultimo_extrato_rel = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
    ultimo_extrato_rel = fdt.workday(data=ultimo_extrato_rel, n_dias=-1)
    #ultimo_extrato =datetime.datetime.date(ultimo_extrato) 
    carteira_rel = rel_off_p.carteira_reliance(ultimo_extrato_rel)   
else: 
    carteira_rel = rel_off_p.carteira_reliance(ultimo_extrato)
    
carteira_rel['ISIN'] = carteira_rel['ISIN'].str.lower()    
 
#Unifica a classificações com a tabela de produtos (ISIN)
carteira_rel=pd.merge(left=carteira_rel,right=produtos_rel, right_on='security',left_on='Nome_produto',how='left').rename(columns={'ISIN_x':'ISIN','asset_class':'subclasse'}).reset_index()
carteira_rel['Nome_produto']=carteira_rel['Nome_produto'].str.lower()

#Junta o campo classe, subclasse e categoria para fazer depara com a classificaçao GPS
carteira_rel['Subcategoria']=carteira_rel['Subcategoria'].fillna('')    
carteira_rel['Classe_Subclasse_Categoria']=carteira_rel['Classe']+'|'+carteira_rel['subclasse']+'|'+carteira_rel['Subcategoria']

NameError: name 'produtos_rel' is not defined

In [23]:
#Traz a posição da primeira carteira da base de dados

ultimo_extrato = data
ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
ultimo_extrato = ultimo_extrato + timedelta(days = -1)

teste = alocacao_por_sc.drop_duplicates()
sc = teste.iloc[0,0]
#sc = '@5AGL1'
df = extrato.posicao_extrato(sc,ultimo_extrato)

if df.empty:
    ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
    ultimo_extrato = ultimo_extrato + timedelta(days = -1)
else:
    ultimo_extrato = ultimo_extrato

df = extrato.posicao_extrato(sc,ultimo_extrato)
df['data_ultimo_extrato']=ultimo_extrato

for i,(index,row) in enumerate(teste.iterrows()):
     if i >0 :
            ultimo_extrato = data
            ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
            ultimo_extrato = ultimo_extrato + timedelta(days = -1)            
            sc = row['new_name']
            df1 = extrato.posicao_extrato(sc,ultimo_extrato)
            if df1.empty:
                ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
                ultimo_extrato = ultimo_extrato + timedelta(days = -1)
            else:
                ultimo_extrato = data
                ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
                ultimo_extrato = ultimo_extrato + timedelta(days = -1)
                ultimo_extrato

            df1 = extrato.posicao_extrato(sc,ultimo_extrato)
            df1['data_ultimo_extrato']=ultimo_extrato          
            df = pd.concat([df,df1])
df.to_excel('carteira_off_gps.xlsx')

#Mapeando as informações da base GPS e linkando com os produtos da base (classificação, isin e ticker do CRM).

#Carteira GPS
carteira_gps = df
carteira_gps=pd.merge(left=carteira_gps,right=produtos_gps,on='IdMysqlProduto',how='left')


In [ ]:
#Fazer a conciliação de contas
conciliacao = df[['NomeContaCrm','NomeSupercarteiraCrm']].drop_duplicates()
conciliacao=conciliacao['NomeSupercarteiraCrm'].str.lower()
todas_sc = alocacao_por_sc[['new_name','new_conta_supercarteiraidname']].drop_duplicates()
conciliacao = pd.merge(left=conciliacao, right=todas_sc,left_on='NomeSupercarteiraCrm',right_on='new_name',how='right')
conciliacao = conciliacao[conciliacao['NomeSupercarteiraCrm'].isnull()]
conciliacao.to_excel('conciliacao.xlsx')

In [ ]:
#Lê o excel que traz todas as classes e subclasses dos fundos (projeto base única de ativos)- Tratamento das informações
classificacoes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='TB Security')
classificacoes.columns =classificacoes.columns.str.strip()
classificacoes = classificacoes[['Ativo_Nome','refClassSubClass','ClassSubClass']].rename(columns={'refAtivo':'Ativo_Nome'})
classificacoes['Ativo_Nome']=classificacoes['Ativo_Nome'].str.lower().apply(lambda x: x.strip())
classificacoes= classificacoes[classificacoes['Ativo_Nome']!='']
classificacoes['refClassSubClass']=classificacoes['refClassSubClass'].astype('str')
classificacoes['refClassSubClass']=classificacoes['refClassSubClass'].apply(lambda x: x.strip())
classificacoes= classificacoes[classificacoes['refClassSubClass']!='']

#Consolida as classificações da base única de ativos com a carteira dos clientes
carteira_rel = pd.merge(left=carteira_rel,right=classificacoes,right_on='Ativo_Nome',left_on='Nome_produto',how='left').drop(columns=['ISIN_y']).rename(columns={'ISIN_x':'ISIN'})
carteira_rel = pd.merge(left=carteira_rel, right=produtos_gps,on='ISIN',how='left').rename(columns={'new_indgroupaxysidname':'Subclasse_gps'})
carteira_rel['Subclasse_gps'] = carteira_rel['Subclasse_gps'].fillna('Não Classificado')
carteira_rel['refClassSubClass'] = carteira_rel['refClassSubClass'].fillna('Não Classificado')
carteira_rel['Classe_gps'] = carteira_rel['Subclasse_gps'].apply(lambda x: dicio_classe_subclasse_gps[x] if x!='Não Classificado' else x)

#Trazendo as classes da GPS

carteira_rel['Classe_gps'] = carteira_rel.apply(lambda row: dicio_classificacao_investimentos_rel[row['refClassSubClass']] if (row['refClassSubClass']!='Não Classificado')&(row['Subclasse_gps'] =='Não Classificado') else row['Classe_gps'],axis=1)
carteira_rel['Classe_gps']=carteira_rel.apply(lambda row: dicio_classe_sub_categoria[row['Classe_Subclasse_Categoria']] if (row['refClassSubClass']=='Não Classificado') &  (row['Classe_gps'] == 'Não Classificado') else row['Classe_gps'],axis=1)
carteira_rel['Classe']=carteira_rel['Classe_gps']
carteira_rel['Subclasse']=carteira_rel['Subclasse_gps']

In [ ]:
#Trazer apenas os clientes da Reliance
#movimentos_rel = carteira_jbfo[carteira_jbfo['Esteira']=='ex-Reliance']


#Verificação de classes aproximadas
verificar_ativos_classificados = carteira_rel.groupby(['Nome_produto','ISIN','refClassSubClass','Classe_Subclasse_Categoria','Classe_gps','Subclasse_gps']).size().reset_index(name='counts')
verificar_ativos_classificados = verificar_ativos_classificados.sort_values(['refClassSubClass','Classe_Subclasse_Categoria','Classe_gps','Subclasse_gps','counts'],ascending=False).groupby(['Classe_gps']).head(5)#.dropna(subset=['classe_sistema'])
verificar_ativos_classificados.to_excel('double_classificacoes.xlsx')

#Trazendo tudo o que é private equity
lista_private_equity = ['investimento','comprometido','desembolsado','desembolsar','investido','investido','investment','commitment','called','pending']
classes=[]
for idx, row in carteira_rel.iterrows():
    if (any(map(row['Nome_produto'].__contains__,lista_private_equity))):
        classe = 'Iliq. P. Equity'
    else:
        classe = row['Classe']
    classes.append(classe)
carteira_rel['Classe']  =  classes  
    

#carteira_rel['Classe'] = carteira_rel.apply(lambda row:'Iliq. P. Equity' if (any(map(row['Nome_produto'].__contains__,lista_private_equity))) else row['Classe'],axis =1)


#Verificar os ativos novos (não classificados)
verificar = carteira_rel[carteira_rel['Classe']=='Não Classificado']
verificar[['Nome_produto','ISIN','refClassSubClass','Classe_Subclasse_Categoria','Classe_gps','Subclasse_gps']].drop_duplicates().to_excel('verificar.xlsx')

# #JUntar a classificação manual com a carteira
# movimentos_rel = pd.merge(left = movimentos_rel, right=ativos_novos, on='Nome_produto',how='left')
# movimentos_rel['classe_sistema'].loc[movimentos_rel['classe_sistema']=='Não Classificado'] = movimentos_rel['Classificação GPS'].loc[movimentos_rel['classe_sistema']=='Não Classificado']
# movimentos_rel=movimentos_rel.drop(columns={'Classificação GPS'})
# movimentos_rel=movimentos_rel[movimentos_rel['classe_sistema']!='não classificado']

In [ ]:
#Unificação das esteiras (Juntar as bases)

carteira_rel['CodigoProduto']=None
carteira_rel['NomeSupercarteiraCrm']=None
carteira_rel['SaldoNaData']=carteira_rel['quantidade']*carteira_rel['PU']
carteira_reliance = carteira_rel[['Data_Posicao','NomeSupercarteiraCrm','Nome_Cliente','Nome_produto','CodigoProduto','ISIN','quantidade','PU','SaldoNaData','moeda','Classe','Subclasse']]
carteira_reliance['Esteira']='ex-Reliance'

carteira_gps['moeda']='NA'
carteira_gps = carteira_gps[['DataPosicao','NomeSupercarteiraCrm','NomeContaCrm','NomeDoProduto','CodigoProduto_x','ISIN','QuantidadeCotas','ValorCota','SaldoNaData','Moeda','Classe', 'Subclasse']].rename(columns={'DataPosicao':'Data_Posicao','NomeContaCrm':'Nome_Cliente','NomeDoProduto':'Nome_produto','QuantidadeCotas':'quantidade','ValorCota':'PU','CodigoProduto_x':'CodigoProduto'})
carteira_gps['Esteira']='ex-Gps'


carteira_jbfo = pd.concat([carteira_reliance,carteira_gps])
carteira_jbfo['Nome_produto']=carteira_jbfo['Nome_produto'].str.lower()
carteira_jbfo['Nome_produto']=carteira_jbfo['Nome_produto'].apply(lambda x: arruma_espaco(x))

In [ ]:
#Filtra os ativos que são imobiliarios e gera uma planilha para consultar na bbg o que é Reit
ativos_imob = carteira_jbfo[carteira_jbfo['Classe']=='Imobiliário'][['Nome_produto','ISIN']]
ativos_imob.to_excel('I:/Shared/SAO_Investimentos/Portfolio Management/Tamara/verificar_reits.xlsx')

#Traz a planilha com as infos da bbg
classificacao_imob = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/Base Real Estate.xlsx')

#Separando os Reits dos Iliquidos
carteira_jbfo = pd.merge(left=carteira_jbfo,right=classificacao_imob,on='ISIN',how='left').drop(columns=['Nome_produto_y']).rename(columns={'Tipo':'Classificacao_Real_Estate'})
carteira_jbfo['Classe'].loc[carteira_jbfo['Classe']=='Imobiliário'] = carteira_jbfo['Classificacao_Real_Estate'].loc[carteira_jbfo['Classe']=='Imobiliário']
carteira_jbfo = carteira_jbfo.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto_x', 'ISIN', 'quantidade','PU']).sort_values(by=['Esteira'])

#Ajuste das classes (padronização dos nomes das classes)
carteira_jbfo['Classe']=carteira_jbfo['Classe'].astype('str')
classes = carteira_jbfo['Classe']
carteira_jbfo['Classe']= [x.replace(' -','') for x in classes]

In [ ]:
#DEixando apenas as colunas necessárias, renomeando as mesmas.
carteira_jbfo = carteira_jbfo[['Data_Posicao','NomeSupercarteiraCrm', 'Nome_Cliente', 'Nome_produto_x','CodigoProduto', 'ISIN', 'quantidade','PU','SaldoNaData', 'moeda', 'Classe', 'Subclasse', 'Esteira']].rename(columns={'Classe':'classe_sistema', 'Subclasse':'subclasse_sistema','Nome_produto_x':'Nome_produto'})
carteira_jbfo = pd.merge(left = carteira_jbfo, right = po_cadastro,right_on ='NomeContaCRM',left_on = 'Nome_Cliente',how='left')
carteira_jbfo = pd.merge(left = carteira_jbfo, right = alocacao_por_sc,right_on ='new_conta_supercarteiraidname',left_on = 'Nome_Cliente',how='left')
carteira_jbfo=carteira_jbfo[['Data_Posicao','NomeSupercarteiraCrm', 'Nome_Cliente', 'Nome_produto','CodigoProduto', 'ISIN', 'quantidade',
        'PU','SaldoNaData', 'moeda', 'classe_sistema', 'subclasse_sistema', 'Esteira',
        'new_politicadeinvestimentoidname','Officer', 'Controller']].rename(columns={'new_politicadeinvestimentoidname':'Politica'})
#carteira_jbfo=carteira_jbfo.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'ISIN', 'quantidade','PU']).sort_values(by=['Esteira'])
carteira_jbfo = carteira_jbfo.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'ISIN', 'quantidade','PU']).sort_values(by=['Esteira'])
carteira_jbfo.to_excel('carteira_off.xlsx')


In [ ]:
#Ajustes na carteira Reliance
analise_carteira_rel = carteira_jbfo[(carteira_jbfo['Esteira']=='ex-Reliance')&(carteira_jbfo['classe_sistema']!='Não Classificado')]
analise_carteira_rel = analise_carteira_rel[analise_carteira_rel['classe_sistema']!='nan']
analise_carteira_rel['classe_asset']=analise_carteira_rel['classe_sistema'].apply(lambda x:dicio_classificacoes[x])
portfolios = novos_mandatos['Portfolio'].drop_duplicates().to_list()
classes_benchs = pd.merge(left=novos_mandatos, right=benchs,left_on='Classe',right_on='De Para - Asset Allocation',how='left').dropna(subset=['Index_Bawm']).drop(columns={'De Para - Asset Allocation'})
#analise_carteira_rel['classe_sistema'].drop_duplicates()

In [ ]:
for portfolio in classes_asset_allocation:
    bench = novos_mandatos[novos_mandatos['Portfolio']==portfolio]
    bench = bench[['Classe','Tático']]

In [ ]:
#Calculando os percentuais:
percentual_produto=[]
financeiro_total_spe =[]
financeiro_total =[]
mandato = []
movimentos_gps=movimentos_gps.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'quantidade','SaldoNaData'])
        #h3=0
        soma = analise_carteira_rel[analise_carteira_rel['Nome_Cliente']==row['Nome_Cliente']]['SaldoNaData'].sum()
        iliquidos = analise_carteira_rel[(analise_carteira_rel['Nome_Cliente']==row['Nome_Cliente'])&(analise_carteira_rel['classe_sistema']=='Iliq. P. Equity')]['SaldoNaData'].sum()
        financeiro_total_spe.append(soma-iliquidos)
        financeiro_total.append(soma)
        percentual= row['SaldoNaData']/(soma-iliquidos)
        percentual_produto.append(percentual) 
analise_carteira_rel['percentual_produto']=percentual_produto
analise_carteira_rel['Financeiro_Total']=financeiro_total
analise_carteira_rel['Financeiro_Total_s/pe']=financeiro_total_spe
     

In [ ]:
#Trazer os movimentos do mandato atual da GPS para os novos mandatos, trazer os preços atuais também
movimentos_gps = carteira_jbfo[carteira_jbfo['Esteira']=='ex-Gps']
movimentos_gps=movimentos_gps.dropna(subset=['classe_sistema'])
movimentos_gps=movimentos_gps.dropna(subset=['classe_sistema'])
movimentos_gps['classe_sistema']=movimentos_gps['classe_sistema'].apply(lambda x:dicio_classificacoes[x])

In [ ]:
#Segregar o que é veiculo JBFO
movimentos_gps=pd.merge(left=movimentos_gps,right=fundos_jbfo,on='CodigoProduto',how='left')
movimentos_gps['Classificação'] = movimentos_gps['Classificação'].astype('str')
movimentos_gps['classe_sistema']=movimentos_gps.apply(lambda row: row['classe_sistema'] if row['Classificação']=='nan' else row['Classificação'],axis=1 )
# movimentos_gps['Financeiro']=movimentos_gps['quantidade']*movimentos_gps['PU']


In [ ]:
#Encontrar o respectivo mandato de cada cliente e verificar os clientes que estão nos mandatos padões
movimentos_gps = pd.merge(left=movimentos_gps,right=mandatos,left_on='Politica',right_on='Anterior',how='left')
movimentos_gps=movimentos_gps.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'quantidade','PU'])
movimentos_gps = movimentos_gps.dropna(subset=['Novo'])

In [ ]:
movimentos_gps['Possui_H3']= movimentos_gps.apply(lambda row: 1 if row['classe_sistema']=='Alternativos Macro Arbitragem - MS - JBFO' else 0,axis=1)
percentual_produto=[]
financeiro_total_spe =[]
financeiro_total =[]
mandato = []
for ind,row in movimentos_gps.iterrows():   
        h3=0
        soma = movimentos_gps[movimentos_gps['NomeSupercarteiraCrm']==row['NomeSupercarteiraCrm']]['SaldoNaData'].sum()
        iliquidos = movimentos_gps[(movimentos_gps['NomeSupercarteiraCrm']==row['NomeSupercarteiraCrm'])&(movimentos_gps['classe_sistema']=='Iliq. P. Equity')]['SaldoNaData'].sum()
        financeiro_total_spe.append(soma-iliquidos)
        financeiro_total.append(soma)
        percentual= row['SaldoNaData']/(soma-iliquidos)
        percentual_produto.append(percentual)
        h3 = movimentos_gps[(movimentos_gps['Nome_Cliente']==row['Nome_Cliente'])&(row['Novo'][-2:]!='MS')]['Possui_H3'].sum()
        if h3>0:
            mandato_fundo = row['Novo']+' MS'
        else:
            mandato_fundo = row['Novo']
        mandato.append(mandato_fundo)        
    
#Incluir no DataFrame o calculo acima
movimentos_gps['percentual_produto']=percentual_produto
movimentos_gps['Financeiro_Total']=financeiro_total
movimentos_gps['Financeiro_Total_s/pe']=financeiro_total_spe
movimentos_gps['Portfolio']=mandato                                            
movimentos_gps = movimentos_gps.sort_values(by=['classe_sistema'])
movimentos_gps = movimentos_gps.fillna(0)
movimentos_gps=movimentos_gps.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'quantidade','SaldoNaData']).drop(columns=['Possui_H3'])

In [ ]:
#Calculando os ajustes necessarios da GPS

# #Deixando apenas as colunas que interessam para o groupby
movimentos_gps = movimentos_gps.groupby(['Data_Posicao','Nome_Cliente','NomeSupercarteiraCrm','classe_sistema','Anterior','Portfolio','Financeiro_Total','Financeiro_Total_s/pe']).sum().reset_index()


In [ ]:
clientes = movimentos_gps['NomeSupercarteiraCrm']
#clientes = ['@5AGL1']
classes = novos_mandatos['Classe'].drop_duplicates()

dataframe = []    
for classe in classes:
    lista = []
    for cliente in clientes:  
        movimentos_gps_selecao=movimentos_gps[movimentos_gps['NomeSupercarteiraCrm']==cliente]
        Data_posicao = movimentos_gps_selecao.iloc[0,0]
        Nome_Cliente = movimentos_gps_selecao.iloc[0,1]
        NomeSupercarteiraCrm = cliente
        classe_sistema = classe
        Anterior = movimentos_gps_selecao.iloc[0,4]
        Portfolio = movimentos_gps_selecao.iloc[0,5]
        Financeiro_Total = movimentos_gps_selecao.iloc[0,6]
        Financeiro_Total_pe = movimentos_gps_selecao.iloc[0,7]
        try:
            Saldo = movimentos_gps_selecao[movimentos_gps_selecao['classe_sistema']==classe_sistema].iat[0,10]
        except:
            Saldo = 0
        try:
            percentual_produto = movimentos_gps_selecao[movimentos_gps_selecao['classe_sistema']==classe_sistema].iat[0,12]
        except:
            percentual_produto = 0
        percentual_proposto= novos_mandatos[(novos_mandatos['Classe']==classe)& (novos_mandatos['Portfolio']==Portfolio)].iat[0,2]
        dataframe.append({'Data_Posicao':Data_posicao,'Nome_Cliente': Nome_Cliente,'NomeSupercarteiraCrm':NomeSupercarteiraCrm, 'classe_sistema': classe_sistema,'PI_Anterior':Anterior,'PI_Proposta':Portfolio,'Financeiro_Total':Financeiro_Total,'Financeiro_s/PE':Financeiro_Total_pe,'Financeiro_Produto':Saldo,'Posicao_Atual':percentual_produto,'Posicao_Proposta':percentual_proposto})  

movimentos_gps = pd.DataFrame.from_dict(dataframe)
movimentos_gps['Movimento']=(movimentos_gps['Posicao_Proposta']-movimentos_gps['Posicao_Atual'])*movimentos_gps['Financeiro_s/PE']            
movimentos_gps=movimentos_gps.fillna(0)            


In [ ]:
#Ajustando a tabela após o groupby e renomendo as colunas
movimentos_gps = pd.pivot_table(data=movimentos_gps,values = ['Posicao_Atual','Posicao_Proposta','Movimento'],columns=['classe_sistema'],index=['Data_Posicao','Nome_Cliente','NomeSupercarteiraCrm','PI_Anterior','PI_Proposta','Financeiro_Total','Financeiro_s/PE'])
movimentos_gps = movimentos_gps.reset_index()
movimentos_gps.columns = movimentos_gps.columns.droplevel(0)
inicio = ['Data_Posicao','SC_Gestão','SC_OFF','Mandato_Anterior','Mandato_Proposto','Financeiro_Total','Financeiro_s/PE']
classe_atual =list(movimentos_gps.columns[-20:])
classes_proposta=list([str(i)+"-Proposto" for i in classes])
classes_movimento=list([str(i)+"-movimento" for i in  classes])
movimentos_gps.columns = inicio + classes_movimento+classe_atual+classes_proposta
movimentos_gps['Alternativos Long Short']=movimentos_gps['Alternativos Long Short']+movimentos_gps['Alternativos Long Short Asia - JBFO']

In [ ]:
classes_jbfo = fundos_jbfo['Classificação'].drop_duplicates().to_list()
classes_jbfo.remove('Alternativos Macro Arbitragem - MS - JBFO')
classes_jbfo.remove('Alternativos Long Short Asia - JBFO')
    
for classe in classes_jbfo:
    lista = []
    lista2 = []
    for ind,row in movimentos_gps.iterrows():
        cliente = row['SC_OFF']
        posicao_classe = row[classe[:-7]]
        posicao_esperada = row[classe[:-7]+'-Proposto']
        posicao_veiculo =  row[classe]
        posicao_classe = posicao_classe + posicao_veiculo
        movimento = posicao_esperada -  posicao_classe
        financeiro = row['Financeiro_s/PE']
        if  movimento>0:
            valor_veiculo = movimento*financeiro
            valor_outros = 0
        elif (movimento < 0) and (posicao_classe*0.95*-1)>movimento:
            valor_veiculo = posicao_veiculo*financeiro*-1
            valor_outros = (posicao_classe-posicao_veiculo)*financeiro*-1
        elif (movimento <0) and (posicao_classe-posicao_veiculo)>(movimento*-1):
            valor_veiculo = 0
            valor_outros = movimento*financeiro
        elif (movimento <0) and ((posicao_classe-posicao_veiculo)==0):
            valor_veiculo = movimento*financeiro
            valor_outros = 0            
        elif row['Alternativos Long Short Asia - JBFO']>0:
            valor_veiculo = (posicao_esperada -posicao_classe)*financeiro
            valor_outros = 0    
        else:
            valor_veiculo = (posicao_esperada-posicao_classe+(posicao_classe-posicao_veiculo))*financeiro
            valor_outros = (movimento*financeiro)- valor_veiculo         
        lista.append(valor_veiculo)
        lista2.append(valor_outros)

    movimentos_gps[classe+'-movimento'] = lista
    movimentos_gps[classe[:-7]+'-movimento'] = lista2
      
 

In [ ]:
#Encontrar os e-mail relacionados à sc
cadastro['OfficerUsuarioU'] = cadastro['OfficerUsuarioU'].str.lower()
cadastro['ControllerUsuarioU'] = cadastro['ControllerUsuarioU'].str.lower()
cadastro['ControllerBKPUsuarioU'] = cadastro['ControllerBKPUsuarioU'].str.lower()
cadastro['DPMUsuarioU'] = cadastro['DPMUsuarioU'].str.lower()
cadastro['DeputyDPMUsuarioU'] = cadastro['DeputyDPMUsuarioU'].str.lower()
movimentos_gps = pd.merge(left=movimentos_gps,right=cadastro,left_on='SC_Gestão',right_on='NomeContaCRM',how='left')
movimentos_gps = pd.merge(left=movimentos_gps,right=distrato,left_on='SC_OFF',right_on='new_name',how='left')
movimentos_gps = movimentos_gps[movimentos_gps['new_contratoname']!='Distrato em Andamento']
movimentos_gps = movimentos_gps.drop(columns=['NomeContaCRM','NomeSuperCarteiraCRM','TipoCliente','new_name','new_contrato','new_contratoname']).drop_duplicates()
        

In [ ]:
movimentos_gps['ControllerBKPUsuarioU'] = movimentos_gps['ControllerBKPUsuarioU'].replace('nd','u45285').replace('na','u45285')
movimentos_gps['DPMUsuarioU'] = movimentos_gps['DPMUsuarioU'].replace('na','u45285').replace('nd','u45285')
movimentos_gps['ControllerBKPUsuarioU'] = movimentos_gps['ControllerBKPUsuarioU'].replace('na','u45285').replace('nd','u45285')
movimentos_gps['DeputyDPMUsuarioU'] = movimentos_gps['ControllerBKPUsuarioU'].replace('na','u45285').replace('nd','u45285')
movimentos_gps['Real Estate-movimento'] = movimentos_gps['Real Estate-movimento']+ movimentos_gps['Real Estate - JBFO-movimento']
movimentos_gps['Real Estate-movimento'] = movimentos_gps['Real Estate-movimento']+ movimentos_gps['Real Estate - JBFO-movimento']


In [ ]:
emails_envio = []
emails_usuarios['UsuarioU']=emails_usuarios['UsuarioU'].str.lower()
for idx, row in movimentos_gps.iterrows(): 
    lista_email=[]
    if row['DPMUsuarioU'] in emails_usuarios['UsuarioU'].to_list():  
        lista_email.append(emails_usuarios.loc[emails_usuarios['UsuarioU']== row['DPMUsuarioU']].iat[0,2])
    if row['OfficerUsuarioU'] in emails_usuarios['UsuarioU'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['UsuarioU']==row['OfficerUsuarioU']].iat[0,2])
    if row['ControllerUsuarioU'] in emails_usuarios['UsuarioU'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['UsuarioU']==row['ControllerUsuarioU']].iat[0,2])
    if row['ControllerBKPUsuarioU'] in emails_usuarios['UsuarioU'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['UsuarioU']==row['ControllerBKPUsuarioU']].iat[0,2]) 
    emails_envio.append(lista_email)
movimentos_gps['emails_aprovadores']= emails_envio 
colunas = movimentos_gps.columns
colunas = [col.replace('-movimento','') for col in colunas]
colunas = [col.replace('-Proposto','') for col in colunas]
movimentos_gps.columns = colunas
movimentos_gps = movimentos_gps.drop(columns={'Iliq. P. Equity','Alternativos Long Short Asia','Alternativos Long Short Asia - JBFO','Real Estate - JBFO'})

In [ ]:
#officers = movimentos_gps['Officer'].drop_duplicates().to_list()
officers = ['Bruna Casanova']
tbl = build_table(mandatos,'blue_dark',text_align = 'center',width="200px") 
tbl2 = build_table(classificacoes,'blue_dark',text_align = 'center',width="200px") 
classes_pm = list(movimentos_gps.columns[7:23])


dicio_veiculos = dict(pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Fundos_Casa')[['Classe','Veiculo Abreviado']].drop_duplicates().values)
       
for officer in officers:
    #caminho_anexo = f'O:/SAO/CMHA_All/Portfolio Management/20 - Offshore/1 - Movimentao_officer/{officer}.xlsx'
    #caminho_anexo = f'C:/Temp/Mov_off/{officer}.xlsx'
    df = movimentos_gps[movimentos_gps['Officer']==officer]
    clientes =df['SC_Gestão']
    sc_off = df['SC_OFF']
    txt_email=[]
    emails = [x for i,elem in df['emails_aprovadores'].items() for x in elem]
    emails=list(set(emails))
    emails = [x for x in emails if x is not None]
    
    txt_inicio = f'''Prezados,<br>   
        Seguem as movimentações propostas para adequação dos portfólios atuais para os novos mandatos. Caso tenham alguma sugestão vinda da equipe dos DPMs, favor desconsiderar.<br><br>    
        Os portfólios que possuem "MS", são aqueles que possuem o H3 Composite (classe Multiestratégia)-<br><br>
        ''' 
    for sc in sc_off:
        
        df = movimentos_gps[movimentos_gps['Officer']==officer]
        df = df[df['SC_OFF']==sc]
        df[df['SC_OFF']==sc][['SC_Gestão']].iloc[0,0]
        if df.empty:
            txt = txt1 = f'<ul><li>Sem movimentações a fazer.</li>'
        else:    
            mandato_atual = df['Mandato_Anterior'].iloc[0]
            mandato_proposto = df['Mandato_Proposto'].iloc[0]
            patrimonio_total = formatar(df['Financeiro_Total'].iloc[0])
            patrimonio_s_pe = formatar(df['Financeiro_s/PE'].iloc[0]) 
            df_financeiro= df.iloc[:, 0:23]
            df_atual= df.iloc[:, 23:39]
            df_proposto=df.iloc[:, 39:55]   
            SC_On = df['SC_Gestão'].iloc[0]
            data_posicao=df.iat[0,0]
            data_posicao = "{}/{}/{}".format(data_posicao.day,data_posicao.month,data_posicao.year)
            txt = f'''<b>Cliente {sc} / {SC_On}:</b><br>
            *Data Posição : {data_posicao},<br>
            * Mandato : {mandato_atual},<br>
            * Mandato Proposto : {mandato_proposto},<br>
            * PL Total : {patrimonio_total},<br>
            * PL desconsiderando Ilíquidos e Imobiliarios iliquidos : {patrimonio_s_pe},<br>'''
            for classe_pm in classes_pm:     
                if classe_pm == 'Alternativos Macro Arbitragem - MS - JBFO':
                    classe_pm_aloc = 'Alternativos Macro Arbitragem - MS - JBFO'
                    ativo = 'H3 Composisite'
                elif classe_pm[-7:]==' - JBFO':
                    classe_pm_aloc = classe_pm[:-7]
                    ativo = dicio_veiculos[classe_pm]
                else:    
                    classe_pm_aloc=classe_pm
                    ativo = dicio_veiculos[classe_pm]            
                percentual_proposto_valor=df_proposto[classe_pm_aloc].iloc[0]*100
                try:
                    percentual_atual_valor=(df_atual[classe_pm_aloc].iloc[0]+df_atual[classe_pm_aloc+' - JBFO'].iloc[0])*100
                except:
                    percentual_atual_valor=df_atual[classe_pm_aloc].iloc[0]*100


                #                 percentual = novos_mandatos[(novos_mandatos['Classe']==classe)&(novos_mandatos['Portfolio']==mandato_proposto)].iat[0,2]*100
                #             else:
                #                 classe=classe_pm
                #                 percentual = novos_mandatos[(novos_mandatos['Classe']==classe)&(novos_mandatos['Portfolio']==mandato_proposto)].iat[0,2]*100
                try:
                    valor = df_financeiro[classe_pm].iloc[0]
                    valor = round(valor,-4)
                    classe = classe_pm
                except:
                    valor=0            
                percentual_atual_valor ='{0:.2f}%'.format(percentual_atual_valor)
                percentual_proposto_valor ='{0:.2f}%'.format(percentual_proposto_valor)
                if valor < 0:
                    valor = abs(valor) 
                    valor=formatar(valor)
                    txt1 = f'<ul><li>Resgate de ${valor} de {ativo} para levar a alocação da classe de {percentual_atual_valor} para o ponto tático de {percentual_proposto_valor}.</li>'
                elif valor > 0:
                    valor = abs(valor)
                    valor=formatar(valor)
                    txt1 = f'<ul><li>Aplicação de ${valor} de {ativo} para levar a alocação da classe de {percentual_atual_valor} para o ponto tático de {percentual_proposto_valor}.</li>'
                else:
                    txt1=''
                txt += txt1 + '</ul>'

            txt_email.append(txt)
        corpo = f"{txt_inicio}{'<br>'.join(txt_email)}"

    subject= f'Movimentos offshore - Novos Mandatos - Base ex-GPS'
    to=['tamara.alves@jbfo.com','portfolio@jbfo.com']+emails
    email = Email(to = to , subject = subject, text= corpo,send = False)     








In [ ]:
colunas = movimentos_gps.columns
colunas = [dicio_veiculos[coluna] if coluna[-7:]==' - JBFO' else coluna for coluna in colunas ] 
movimentos_gps.colums = colunas

for officer in officers:
    df = movimentos_gps[movimentos_gps['Officer']==officer]

    df = df.reindex().drop_duplicates()  

    df.to_excel(f'O:/SAO/CMHA_All/Portfolio Management/20 - Offshore/1 - Movimentao_officer/{officer}.xlsx')

In [29]:
#Pegando o primeiro dia de cada mes
data = fdt.hoje()
data = data - relativedelta(days=30)
data = date(data.year, data.month, 1) 
data=pd.to_datetime(data)

#Carteira do ultimo extrato
alocacao = rel_off_c.carteira_por_peso(data)
alocacao['NomeClasse']=alocacao['NomeClasse'].str.lower()

#dicionario de classes - Deixando tudo minusculo e equiparando com as classes de asset allocation
dicio_classes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classes=dicio_classes[['Classe_Alocacao_sistema','Classe GPS']]
dicio_classes = dict(dicio_classes.values)

del dicio_classes[np.nan]
dicio_classes = dict((k.lower(), v.lower()) for k, v in dicio_classes.items()) 
alocacao['NomeClasse']=alocacao['NomeClasse'].apply(lambda x:dicio_classes[x] )
alocacao=alocacao.dropna(subset=['nome_reduzido'])


del dicio_classificacoes[np.nan]
dicio_classificacoes = dict((k.lower(), v.lower()) for k, v in dicio_classificacoes.items()) 
alocacao['NomeClasse']=alocacao['NomeClasse'].apply(lambda x:dicio_classificacoes[x])

#Fazendo o merge com todos os possíveis portfolios
#novos_mandatos['Classe'] = novos_mandatos['Classe'].str.lower()
novos_mandatos = novos_mandatos[novos_mandatos['Classe']!='alternativos long short asia']
novos_mandatos = novos_mandatos[novos_mandatos['Portfolio'].str.contains('MS') == False]

#Puxar os clientes para o for
clientes = alocacao['nome_reduzido'].drop_duplicates().to_list()
clientes = [x for x in clientes if x is not None]

#Puxar os portfolios para o for
portfolios = novos_mandatos['Portfolio'].drop_duplicates().to_list()

#dicionario de classes - Deixando tudo minusculo e equiparando com as classes de asset allocation
dicio_classes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classes=dicio_classes[['Classe_Alocacao_sistema','Classe GPS']]
dicio_classes = dict(dicio_classes.values)

#Rentabilidade das carteiras
rentabilidade = rel_off_c.rentabilidade(data)
rentabilidade = rentabilidade.loc[rentabilidade.groupby('nome_reduzido').DataProcessamentoFim.idxmin()].reset_index(drop=True)
rentabilidade = rentabilidade[['nome_reduzido','PontosRisco','RetornoLiquido_NoAno','RetornoLiquido_12Meses']]

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [26]:
dicio_classes

{'Ações': 'RV',
 'Ações Brasil': 'RV',
 'Caixa': 'RF CP',
 'Fundo de Crédito Total Return': 'RF Livre',
 'Fundos': 'Alt Multistr',
 'Instrumentos emitidos/garantidos por governos G7': 'RF Glob',
 'Investimentos de Curto Prazo': 'RF CP',
 'Moedas': 'Outros',
 'Outros Investimentos': 'Outros',
 'Real Estate': 'Imobiliário',
 'Renda Fixa Brasil': 'RF Livre',
 'Renda Fixa Crédito': 'RF Livre',
 'Soberanos Brasil': 'RF Livre',
 'Soberanos Investment Grade': 'RF Glob',
 nan: nan}

In [30]:
pos = pd.DataFrame()
tracking= []
clientes = ['REPAR_JB']

for cliente in clientes:
    nome=cliente
    df = alocacao[alocacao['nome_reduzido']==nome]
    df = df[df['DataProcessamentoFim'] == df['DataProcessamentoFim'].max()]
    df = df.groupby(['nome_reduzido','NomeClasse'])[['PcInicioAno']].sum().reset_index()
    
    df= pd.merge(left=df,right=novos_mandatos,left_on='NomeClasse',right_on='Classe',how='right')
    df = df[df['Classe'].str.contains('jbfo') == False]
    df['nome_reduzido']=df['nome_reduzido'].fillna(nome)
    df['PcInicioAno']= df['PcInicioAno'].fillna(0)
    df = df.rename(columns={'PcInicioAno':'Atual'})    
    df= df[['Portfolio','Classe','Tático','Atual']]
    
#     for portfolio in portfolios:
#         analise = df[df['Portfolio']==portfolio]
#         analise = analise[['Classe','Tático','Atual']]
#         te=calc.tracking_error(analise)
#         tracking.append({'Cliente': nome, 'portfolio': portfolio,'TE': te})
        
    


In [31]:
df

,Portfolio,Classe,Tático,Atual
0,Core Blend,Alternativos Crédito,0.025,0.0
1,Core Blend,Alternativos Crédito - JBFO,0.000,0.0
2,Core Blend,Alternativos Long Short,0.035,0.0
3,Core Blend,Alternativos Long Short - JBFO,0.000,0.0
4,Core Blend,Alternativos Long Short Asia,0.000,0.0
...,...,...,...,...
135,Focus Portfolio 4,Renda Fixa Crédito - JBFO,0.000,0.0
136,Focus Portfolio 4,Renda Fixa Global,0.175,0.0
137,Focus Portfolio 4,Renda Fixa Global - JBFO,0.000,0.0
138,Focus Portfolio 4,Renda Variável Global,0.500,0.0


In [ ]:
#port_recomendado.to_excel('mandato_proposto.xlsx')
calc.vols